In [6]:
!pip install -U tensorflow_datasets

Requirement already up-to-date: tensorflow_datasets in /usr/local/lib/python3.6/dist-packages (1.0.2)


In [7]:
from __future__ import absolute_import, division, print_function

# Import tf and tf datasets
import tensorflow as tf
import tensorflow_datasets as tfds
tf.logging.set_verbosity(tf.logging.ERROR)

# Helper lib
import math
import numpy as np
import matplotlib.pyplot as plt

# Improve progress bar display
import tqdm
import tqdm.auto
tqdm.tqdm = tqdm.auto.tqdm

print(tf.__version__)

tf.enable_eager_execution()
tfds.disable_progress_bar()

1.14.0


In [0]:
class FashionMNIST_NN(object):
  def __init__(self, model=None):
    self.class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
             'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']
    if model:
      self.model = model
    else:
      self.model = model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
        tf.keras.layers.Dense(128, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
      ])
    
    # Load data
    self.dataset, self.metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
    self.train_dataset, self.test_dataset = self.dataset['train'], self.dataset['test']
    
    # normalize
    self.train_dataset = self.train_dataset.map(self.normalize)
    self.test_dataset = self.test_dataset.map(self.normalize)
    
    # Number of examples
    self.num_train_examples = self.metadata.splits['train'].num_examples
    self.num_test_examples = self.metadata.splits['test'].num_examples

  
  def normalize(self, img, labels):
    img = tf.cast(img, tf.float32)
    img /= 255
    return img, labels
   
  def set_model(self, model):
    self.mode = model
   
  def print_data_size(self):
    print('Number of training examples: {}'.format(self.num_train_examples))
    print('Number of test examples: {}'.format(self.num_test_examples))
   
  def test_model(self, e=5, batch_size=32):
    print('Testing model with {} epochs and {} batch size.'.format(e, batch_size))
    
    # dividing into batches
    train_dataset = self.train_dataset.repeat().shuffle(self.num_train_examples).batch(batch_size)
    test_dataset = self.test_dataset.batch(batch_size)

    # Compile and Fit the model
    self.model.compile(
      optimizer='adam',
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy']
    )
    self.model.fit(train_dataset, epochs=e, steps_per_epoch=math.ceil(self.num_train_examples/batch_size), verbose=False)

    # Computing accuracy
    test_loss, test_accuracy = self.model.evaluate(test_dataset, steps=math.ceil(self.num_test_examples/batch_size))
    #print('\n')

In [0]:
FMNIST = FashionMNIST_NN()

In [26]:
FMNIST.test_model()

Testing model with 5 epochs and 32 batch size.
313/313 [==============================] - 2s 7ms/step - loss: 0.3708 - acc: 0.8695


In [0]:
FMNIST.set_model(tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu, input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2,2), strides=2),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2,2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
]))

In [28]:
FMNIST.test_model()

Testing model with 5 epochs and 32 batch size.
313/313 [==============================] - 2s 7ms/step - loss: 0.3345 - acc: 0.8857
